# Feature engineering
Nesse notebook vamos tratar os dados dos jogadores e criar features novas com bases nos dados que temos

In [2]:
from database.bq_connection import execute
import pandas as pd

In [40]:
query = """SELECT * FROM `puc-sp.fifa.player`"""

data = execute(query, "C:\\Users\\gugat\\fifa22-players-analysis\\database\\filekey.key")
print(data.shape)
data

(3888, 90)


,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,dob,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,GK,76,76,NaN,NaN,1987-01-27,...,29+2,29+2,29+2,27+2,27+2,28+2,28+2,28+2,27+2,75+1
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,GK,76,76,NaN,NaN,1987-10-23,...,37+2,37+2,37+2,31+2,31+2,35+2,35+2,35+2,31+2,75+1
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,GK,74,74,NaN,NaN,1988-03-17,...,22+2,22+2,22+2,19+2,18+2,22+2,22+2,22+2,18+2,73+1
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,GK,72,72,NaN,NaN,1993-03-21,...,27+2,27+2,27+2,26+2,26+2,27+2,27+2,27+2,26+2,71+1
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,GK,72,82,NaN,NaN,1999-02-25,...,24+2,24+2,24+2,19+2,19+2,25+2,25+2,25+2,19+2,71+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,169588,https://sofifa.com/player/169588/jonny-evans/2...,J. Evans,Jonathan Grant Evans,CB,80,80,10000000.0,95000.0,1988-01-03,...,76+2,76+2,76+2,71+2,73+2,80,80,80,73+2,21+2
3884,232119,https://sofifa.com/player/232119/caglar-soyunc...,Ç. Söyüncü,Çağlar Söyüncü,CB,80,83,25000000.0,88000.0,1996-05-23,...,76+2,76+2,76+2,73+2,74+2,80+2,80+2,80+2,74+2,14+2
3885,222501,https://sofifa.com/player/222501/timothy-casta...,T. Castagne,Timothy Castagne,"RB, RM",79,82,21500000.0,84000.0,1995-12-05,...,75+2,75+2,75+2,77+2,77+2,73+2,73+2,73+2,77+2,19+2
3886,202849,https://sofifa.com/player/202849/jannik-vester...,J. Vestergaard,Jannik Vestergaard,CB,78,79,13500000.0,83000.0,1992-08-03,...,74+2,74+2,74+2,64+2,66+2,78+1,78+1,78+1,66+2,20+2


Nesse dataset temos apenas `dob` (data de nascimento) dos jogadores, vamos adicionar uma coluna com as idades dos jogadores

In [35]:
from datetime import date, datetime
current_date = datetime.today()

data.dob = pd.to_datetime(data.dob)

def age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

data["age"] = data.dob.apply(age)
data[["dob", "age"]].head()

,dob,age
0,1987-01-27,35
1,1987-10-23,34
2,1988-03-17,34
3,1993-03-21,29
4,1999-02-25,23


Outra tratativa a ser feita é transformar cada `player_traits` e cada `player_tags` diferente em uma coluna boleana

In [36]:
data[["player_traits", "player_tags", "player_positions"]]

,player_traits,player_tags,player_positions
0,Team Player,NaN,GK
1,"Leadership, Comes For Crosses",NaN,GK
2,NaN,NaN,GK
3,"GK Long Throw, Cautious With Crosses",NaN,GK
4,NaN,NaN,GK
...,...,...,...
19234,NaN,NaN,LW
19235,NaN,NaN,RW
19236,NaN,NaN,ST
19237,NaN,NaN,ST


In [37]:
player_traits_dummies = data.player_traits.str.get_dummies(sep=',')
player_tags_dummies = data.player_tags.str.get_dummies(sep=',')
player_positions_dummy = data.player_positions.str.get_dummies(sep=', ')

processed_data = pd.concat([data, player_traits_dummies, player_tags_dummies, player_positions_dummy], axis=1)
processed_data.drop(columns=["player_traits", "player_tags", "player_positions"], inplace=True)
processed_data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,...,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,...,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,...,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,...,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,...,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,...,1,0,0,0,0,0,0,0,0,0


In [42]:
# Convertendo o tipo das colunas
columns_to_evaluate = "ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk".split(",")
for column_to_evaluate in columns_to_evaluate:
    evaluated_column = data[column_to_evaluate].apply(lambda cell: eval(cell))
    data[column_to_evaluate] = evaluated_column

In [44]:
processed_data.to_csv("processed_data.csv", index=False)

In [39]:
processed_data.shape

(19239, 184)